# Input-Dependency Thermodynamics: The Physics of Language

**Date:** 2026-01-05
**Goal:** Prove that residual stream gain is INPUT-DEPENDENT, not architecture-fixed

## The Discovery

LLaMA 3.1 showed:
- **0.48x contraction** with prompt: "The capital of France is"
- **1.53x expansion** with prompt: "The quick brown fox jumps over the lazy dog."

**Difference: 3.19x** - Same model, different prompts!

## Hypothesis: Thermodynamics of Difficulty

```
Factual (Trivial)   → Contraction (Model is "sure", low entropy)
Syntactic (Grammar) → Contraction? (Grammar constrains, reduces search space)
Ambiguous (Open)    → Expansion (Model needs to "explore", high entropy)
Nonsense (Chaos)    → Maximum Expansion (Confusion/uncertainty)
```

**Physical Analogy:** Car with automatic transmission
- Cruising (easy) → Low RPM, high gear (contraction)
- Hill climbing (hard) → Kickdown, high RPM (expansion)

In [ ]:
# Cell 0: HuggingFace Login
from huggingface_hub import login
login()
print("HuggingFace login complete!")

In [ ]:
# Cell 1: Setup & Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer
from scipy.stats import entropy
import json
import os

# Configure plots
plt.style.use('seaborn-v0_8-paper')
sns.set_context("talk")

RESULTS_DIR = './Results'
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Cell 2: The "Difficulty" Spectrum
# 4 types of prompts with increasing uncertainty

PROMPTS = {
    "1_Factual": {
        "text": "The capital of France is",
        "difficulty": "Trivial",
        "expected": "CONTRACTION",
        "reason": "Single correct answer, model is certain"
    },
    "2_Syntactic": {
        "text": "The agreement, which, notwithstanding the fact that it was signed only yesterday, effectively binds all parties immediately, stipulates that",
        "difficulty": "Nested Grammar",
        "expected": "CONTRACTION (Hypothesis!)",
        "reason": "Grammar constrains, search space is LIMITED by syntax"
    },
    "3_Ambiguous": {
        "text": "The true meaning of happiness is often found in",
        "difficulty": "Open-ended",
        "expected": "EXPANSION",
        "reason": "Many valid continuations, high entropy"
    },
    "4_Nonsense": {
        "text": "Table sky run blue jump quickly under over",
        "difficulty": "Chaos",
        "expected": "MAXIMUM EXPANSION",
        "reason": "No grammar, no semantics, pure confusion"
    },
    "5_Original_Test": {
        "text": "The quick brown fox jumps over the lazy dog.",
        "difficulty": "Moderate (reference)",
        "expected": "~1.53x (as measured)",
        "reason": "The original LLaMA 2 vs 3.1 test prompt"
    }
}

print("Prompt Spectrum:")
print("="*70)
for name, info in PROMPTS.items():
    print(f"\n{name}:")
    print(f"  Text: '{info['text'][:50]}...' " if len(info['text']) > 50 else f"  Text: '{info['text']}'")
    print(f"  Difficulty: {info['difficulty']}")
    print(f"  Expected: {info['expected']}")

In [ ]:
# Cell 3: Residual Stream + Entropy Analyzer

class ThermodynamicsAnalyzer:
    """Analyze residual stream dynamics AND output entropy."""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = next(model.parameters()).device
        self.hooks = []
        self.residual_norms = []
        self.embedding_norm = None
    
    def _make_embedding_hook(self):
        def hook(module, args, output):
            with torch.no_grad():
                self.embedding_norm = output.float().norm().item()
        return hook
    
    def _make_layer_hook(self, layer_idx):
        def hook(module, args, output):
            if isinstance(output, tuple):
                hidden = output[0]
            else:
                hidden = output
            with torch.no_grad():
                norm = hidden.float().norm().item()
                self.residual_norms.append((layer_idx, norm))
        return hook
    
    def register_hooks(self):
        # Embedding hook
        h_emb = self.model.model.embed_tokens.register_forward_hook(self._make_embedding_hook())
        self.hooks.append(h_emb)
        
        # Layer hooks
        for i, layer in enumerate(self.model.model.layers):
            h = layer.register_forward_hook(self._make_layer_hook(i))
            self.hooks.append(h)
    
    def remove_hooks(self):
        for h in self.hooks:
            h.remove()
        self.hooks = []
    
    def clear(self):
        self.residual_norms = []
        self.embedding_norm = None
    
    def analyze(self, prompt):
        """Full analysis: residual gains + output entropy."""
        self.clear()
        
        # Tokenize
        inputs = self.tokenizer(prompt, return_tensors='pt').to(self.device)
        
        # Forward pass
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
        
        # Compute gains
        sorted_norms = sorted(self.residual_norms, key=lambda x: x[0])
        all_norms = [('emb', self.embedding_norm)] + sorted_norms
        
        gains = []
        norms = [n for _, n in all_norms]
        
        for i in range(1, len(all_norms)):
            prev_norm = all_norms[i-1][1]
            curr_norm = all_norms[i][1]
            if prev_norm > 1e-8:
                gain = curr_norm / prev_norm
            else:
                gain = 1.0
            gains.append(float(gain))
        
        # Compute output entropy (Shannon entropy of softmax over last token logits)
        last_logits = logits[0, -1, :]
        probs = torch.softmax(last_logits, dim=0).cpu().numpy()
        token_entropy = entropy(probs)  # Shannon entropy in nats
        
        # Top 5 most likely tokens
        top_k = 5
        top_probs, top_indices = torch.topk(torch.tensor(probs), top_k)
        top_tokens = [self.tokenizer.decode([idx]) for idx in top_indices.tolist()]
        
        return {
            'prompt': prompt,
            'gains': gains,
            'norms': norms,
            'last_gain': gains[-1] if gains else 0,
            'expands': gains[-1] > 1.0 if gains else False,
            'entropy': float(token_entropy),
            'top_tokens': list(zip(top_tokens, top_probs.tolist())),
            'cumulative_energy': float(np.prod(gains)) if gains else 0
        }

print("ThermodynamicsAnalyzer defined")

In [ ]:
# Cell 4: Load LLaMA 3.1

MODEL_NAME = 'meta-llama/Llama-3.1-8B'

print(f"Loading {MODEL_NAME}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map='auto',
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print(f"Model loaded: {MODEL_NAME}")
print(f"Layers: {len(model.model.layers)}")

In [ ]:
# Cell 5: Run the Thermodynamics Experiment

analyzer = ThermodynamicsAnalyzer(model, tokenizer)
analyzer.register_hooks()

results = {}

print("="*80)
print("THERMODYNAMICS OF DIFFICULTY")
print("="*80)
print(f"\n{'Type':<20} | {'Entropy':>8} | {'Last Gain':>10} | {'Behavior':>12} | Top Token")
print("-" * 80)

for name, info in PROMPTS.items():
    text = info['text']
    
    # Analyze
    res = analyzer.analyze(text)
    res['prompt_type'] = name
    res['difficulty'] = info['difficulty']
    res['expected'] = info['expected']
    results[name] = res
    
    # Interpret
    gain = res['last_gain']
    behavior = "EXPANDS" if gain > 1.0 else "CONTRACTS"
    ent = res['entropy']
    top_tok = res['top_tokens'][0][0] if res['top_tokens'] else "?"
    
    print(f"{name:<20} | {ent:>8.2f} | {gain:>9.3f}x | {behavior:>12} | '{top_tok}'")

analyzer.remove_hooks()
print("\nAnalysis complete!")

In [ ]:
# Cell 6: Correlation Analysis - Entropy vs Gain

from scipy.stats import pearsonr, spearmanr

entropies = [results[k]['entropy'] for k in PROMPTS.keys()]
last_gains = [results[k]['last_gain'] for k in PROMPTS.keys()]

# Compute correlations
pearson_r, pearson_p = pearsonr(entropies, last_gains)
spearman_r, spearman_p = spearmanr(entropies, last_gains)

print("="*60)
print("ENTROPY vs GAIN CORRELATION")
print("="*60)
print(f"\nPearson r:  {pearson_r:.4f} (p={pearson_p:.4f})")
print(f"Spearman r: {spearman_r:.4f} (p={spearman_p:.4f})")

print("\n" + "="*60)
if pearson_r > 0.5 and pearson_p < 0.1:
    print("HYPOTHESIS SUPPORTED!")
    print("="*60)
    print("\nPositive correlation between output entropy and last-layer gain!")
    print("Energy Expenditure ∝ Uncertainty")
    print("\nPhysical Law: The model 'works harder' (expands) when uncertain.")
elif pearson_r < -0.5 and pearson_p < 0.1:
    print("INVERSE CORRELATION!")
    print("="*60)
    print("\nNegative correlation: Low entropy → High gain?")
    print("This would suggest the model 'broadcasts' certainty.")
else:
    print("INCONCLUSIVE")
    print("="*60)
    print(f"\nCorrelation r={pearson_r:.2f} is not strong enough.")
    print("Relationship may be non-linear or other factors matter.")

In [ ]:
# Cell 7: Visualization

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Color coding by difficulty
colors = {
    '1_Factual': 'green',
    '2_Syntactic': 'blue', 
    '3_Ambiguous': 'orange',
    '4_Nonsense': 'red',
    '5_Original_Test': 'purple'
}

# Plot 1: Layer-wise Gains by Prompt Type
ax1 = axes[0, 0]
for name, res in results.items():
    gains = res['gains']
    ax1.plot(gains, label=f"{name} (H={res['entropy']:.1f})", 
             color=colors[name], linewidth=2, marker='o', markersize=3, alpha=0.7)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='Neutral (Gain=1)')
ax1.set_xlabel('Layer')
ax1.set_ylabel('Gain (||x_{l+1}|| / ||x_l||)')
ax1.set_title('Residual Stream Gain by Prompt Type\n"Energy Flow" Through Layers')
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3)

# Plot 2: Last Gain vs Entropy (The Key Relationship!)
ax2 = axes[0, 1]
for name, res in results.items():
    ax2.scatter(res['entropy'], res['last_gain'], s=200, color=colors[name], 
                label=name, alpha=0.8, edgecolors='black')
ax2.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Output Entropy (nats)')
ax2.set_ylabel('Last Layer Gain')
ax2.set_title(f'Entropy vs Gain\n(Pearson r={pearson_r:.2f}, p={pearson_p:.3f})')
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(entropies, last_gains, 1)
p = np.poly1d(z)
x_line = np.linspace(min(entropies), max(entropies), 100)
ax2.plot(x_line, p(x_line), 'k--', alpha=0.5, label='Trend')

# Plot 3: Bar Chart - Last Gains by Type
ax3 = axes[1, 0]
names = list(results.keys())
gains = [results[k]['last_gain'] for k in names]
bar_colors = [colors[k] for k in names]
bars = ax3.bar(range(len(names)), gains, color=bar_colors, alpha=0.8, edgecolor='black')
ax3.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
ax3.set_xticks(range(len(names)))
ax3.set_xticklabels([n.split('_')[1] for n in names], rotation=30, ha='right')
ax3.set_ylabel('Last Layer Gain')
ax3.set_title('Last Layer Gain by Prompt Type\n(>1 = Expansion, <1 = Contraction)')
for bar, val in zip(bars, gains):
    ax3.annotate(f'{val:.2f}x', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                 ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 4: Entropy Bar Chart
ax4 = axes[1, 1]
ents = [results[k]['entropy'] for k in names]
bars = ax4.bar(range(len(names)), ents, color=bar_colors, alpha=0.8, edgecolor='black')
ax4.set_xticks(range(len(names)))
ax4.set_xticklabels([n.split('_')[1] for n in names], rotation=30, ha='right')
ax4.set_ylabel('Output Entropy (nats)')
ax4.set_title('Output Entropy by Prompt Type\n(Higher = More Uncertain)')
for bar, val in zip(bars, ents):
    ax4.annotate(f'{val:.1f}', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                 ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/input_dependency_thermodynamics.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nVisualization saved to {RESULTS_DIR}/input_dependency_thermodynamics.png")

In [ ]:
# Cell 8: The Schachtelsatz Paradox Test

print("="*70)
print("THE SCHACHTELSATZ PARADOX")
print("="*70)

print("\nHypothesis:")
print("  For HUMANS: Nested sentences are HARD (working memory overload)")
print("  For LLMs:   Nested sentences may be EASY (grammar constrains search)")

# Compare Syntactic vs Factual
syntactic = results['2_Syntactic']
factual = results['1_Factual']

print(f"\nResults:")
print(f"  Factual:   Gain={factual['last_gain']:.3f}x, Entropy={factual['entropy']:.2f}")
print(f"  Syntactic: Gain={syntactic['last_gain']:.3f}x, Entropy={syntactic['entropy']:.2f}")

print("\n" + "="*70)
if syntactic['last_gain'] < 1.0 and factual['last_gain'] < 1.0:
    ratio = syntactic['last_gain'] / factual['last_gain']
    if abs(ratio - 1.0) < 0.3:
        print("SCHACHTELSATZ PARADOX CONFIRMED!")
        print("="*70)
        print("\nBoth Syntactic and Factual prompts cause CONTRACTION.")
        print("Grammar = Constraint = Low Entropy = Easy for LLM")
        print("\n→ LLM Difficulty ≠ Human Difficulty!")
    else:
        print("BOTH CONTRACT, but different magnitudes.")
elif syntactic['last_gain'] > 1.0:
    print("SYNTACTIC EXPANDS - Grammar does NOT constrain!")
else:
    print("Mixed results - investigate further.")

In [ ]:
# Cell 9: Save Results

final_results = {
    'experiment': 'Input-Dependency Thermodynamics',
    'date': '2026-01-05',
    'model': MODEL_NAME,
    'hypothesis': 'Residual stream gain is input-dependent, not architecture-fixed',
    'discovery': 'LLaMA 3.1 showed 0.48x (factual) vs 1.53x (complex) - 3.19x difference!',
    'prompts': {k: {'text': v['text'], 'difficulty': v['difficulty']} for k, v in PROMPTS.items()},
    'results': {k: {
        'last_gain': v['last_gain'],
        'entropy': v['entropy'],
        'expands': v['expands'],
        'gains': v['gains'],
        'top_tokens': v['top_tokens']
    } for k, v in results.items()},
    'correlation': {
        'pearson_r': pearson_r,
        'pearson_p': pearson_p,
        'spearman_r': spearman_r,
        'spearman_p': spearman_p
    },
    'conclusion': 'PENDING'
}

# Determine conclusion
if pearson_r > 0.5:
    final_results['conclusion'] = 'CONFIRMED: Energy ∝ Uncertainty'
elif pearson_r < -0.5:
    final_results['conclusion'] = 'INVERSE: Energy ∝ Certainty'
else:
    final_results['conclusion'] = 'COMPLEX: Non-linear relationship'

output_path = f'{RESULTS_DIR}/input_dependency_thermodynamics.json'
with open(output_path, 'w') as f:
    json.dump(final_results, f, indent=2, default=str)

print(f"Results saved to {output_path}")

In [ ]:
# Cell 10: Download Results

from google.colab import files

print("="*70)
print("DOWNLOADING RESULTS...")
print("="*70)

for filepath in [f'{RESULTS_DIR}/input_dependency_thermodynamics.json',
                 f'{RESULTS_DIR}/input_dependency_thermodynamics.png']:
    if os.path.exists(filepath):
        print(f"Downloading: {filepath}")
        files.download(filepath)

print("\nDownload complete!")

## Expected Results

### If Hypothesis Confirmed:
```
Factual ("The capital of France is"):    LOW entropy, CONTRACTS
Syntactic (nested sentence):             LOW entropy, CONTRACTS (Schachtelsatz Paradox!)
Ambiguous ("meaning of happiness"):      HIGH entropy, EXPANDS
Nonsense (random words):                 HIGHEST entropy, MAXIMUM EXPANSION

Correlation: Entropy ↔ Gain (positive)
```

### Physical Law (if confirmed):
```
Energy Expenditure = f(Uncertainty)

The model "works harder" (expands residual stream) when it's unsure.
The model "relaxes" (contracts residual stream) when it knows the answer.
```

### Implications for Paper #3:
- Residual stream dynamics are NOT just architecture-dependent
- They are STATE-dependent (input determines behavior)
- The "0.48x vs 1.53x" mystery is SOLVED: Different prompts!
- New dimension: Static Architecture × Dynamic State